In [1]:
%pip install -Uqqq pip --progress-bar off
%pip install -qqq ollama --progress-bar off
%pip install -qqq pathlib --progress-bar off
%pip install -qqq pandas --progress-bar off
%pip install -qqq PyPDF2 --progress-bar off
%pip install -qqq owlready2 --progress-bar off
%pip install -qqq rdflib --progress-bar off
%pip install -qqq langchain-ollama --progress-bar off
%pip install -qqq langchain-community --progress-bar off
%pip install -qqq langchain_community pypdf --progress-bar off
%pip install -qqq langchain-chroma --progress-bar off


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
#all the imports
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import ollama
import json
from enum import Enum
MODEL = "llama3.1:8b-instruct-q8_0"

In [3]:
# Load the PDF
pdf_loader = PyPDFLoader(file_path="sensors.pdf")
docs = pdf_loader.load();


In [4]:
# Calculate Average Document Size
def word_count(texts):
  total_words = 0
  total_characters = 0

  for doc in texts:
      content = doc.page_content
      word_count = len(content.split())  # Count words
      char_count = len(content)         # Count characters
      total_words += word_count
      total_characters += char_count

  num_docs = len(texts)
  avg_words = total_words / num_docs if num_docs > 0 else 0
  avg_characters = total_characters / num_docs if num_docs > 0 else 0

  print(f"Average words per document: {avg_words}")
  print(f"Average characters per document: {avg_characters}")

word_count(docs)

Average words per document: 438.2
Average characters per document: 2702.8


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

word_count(splits)

Average words per document: 136.78666666666666
Average characters per document: 843.32


In [6]:
#create embeddings and store in CHROMA database
embeddings = OllamaEmbeddings(model=MODEL)
vector_store = Chroma(
    collection_name="KGLLM",
    embedding_function=embeddings,
)
# Add documents and their embeddings to Chroma (might take some time)
vector_store.add_documents(documents=splits)

['a7433a04-d412-426d-96bd-eb839a57a76e',
 'c59d4f58-ee9a-4cf5-9720-affe416f4cbc',
 '7d9ffa22-6673-4692-9acc-fa19fc757819',
 '4110b3ac-d7cc-43ef-9cf2-633bfa97f075',
 '7efafda7-051d-4ade-bb6e-89bf8b42503e',
 '919223db-bf86-4503-a20c-036a3e074c72',
 '8b6fbdf6-73b5-4bbf-a938-ccc840def8bb',
 'c1791900-fd6b-4e8c-b176-68538b7312e5',
 '0290ee9d-442b-4c3b-8149-cc2788426714',
 '20843007-6691-459b-aba2-d84e0fdb57f8',
 '59f35655-2279-4ef5-958b-027f3de626bf',
 'd4b45e65-fb1f-4682-b49f-56cb64d7aa3c',
 'fa186c3f-877d-4d49-b6eb-f1cb17238996',
 '61be8e98-b9f2-4716-a5bb-c974752a0651',
 'faf9e713-bef6-41ba-9442-c7c98a5452bc',
 'cda6226a-093f-4134-86e8-a1fbd483e05f',
 'a6481883-a493-4460-8148-f8db3273f8b8',
 'd98dae45-1668-466b-8e3c-45748a12b073',
 '9441a65b-406f-42cc-8969-d27d0cf90759',
 '985b9d10-cc86-4c65-85d2-26bce26bf63f',
 '7e836337-632a-44e3-9f82-00773c01d9df',
 '3ff6a170-b776-4751-8999-fa8622385499',
 '55881878-6b2b-46ab-b446-4dabe70fc2ca',
 '00ef9f08-85ff-41ca-9c61-ee3e380f029d',
 'dc5511d1-e558-

In [56]:
#query the database
query = """
Extract different LiDAR sensors, their categories, and detailed parts from the given text. 
Additionally, identify and describe the relationships between each sensor and its parts, 
including any specific functions or configurations of the parts in relation to the sensors.
"""
retrieved_docs = vector_store.similarity_search(query, k=15)
# Concatenate the retrieved chunks
retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
# #save the retrieved text
# with open("retrieved_text.txt", "w", encoding="utf-8") as f:
#     f.write(retrieved_text)

In [57]:
SUMMARIZE_PROMPT = """
As a LiDAR sensor expert, your task is to extract, categorize, and group all the **real, specific, and branded** LiDAR sensor names and their associated components (parts) mentioned in the following text.

**Output Requirements**:
1. Provide a concise and structured list of LiDAR sensors and their associated parts.
2. Group and standardize similar categories to avoid duplicates (e.g., "Automotive LiDAR", "Automotive LiDAR Sensors", and "Automotive Sensors" should be grouped under one category, "Automotive LiDAR").
3. Categorize the sensors based on their application, type, or any mentioned specifications (e.g., Automotive, Industrial, Surveying).
4. Identify and list the **specific parts or components** of each sensor (e.g., "Laser Module", "Spinning Mechanism", "Control Unit"). If no parts are mentioned for a sensor, leave the "parts" field empty.

**Exclusion Criteria**:
- Do not include generic terms such as "LiDAR", "LiDAR sensor", "LiDAR sensors", "various LiDAR sensors", or "no specific brand model mentioned".
- Exclude datasets or evaluation tools (e.g., "Kitti", "NuScenes").
- Ignore descriptions or summaries like "solid-state LiDAR", "lidar technology", or "selected sensors are mechanical or solid-state types".
- Avoid placeholder entries like "not mentioned" or "various".

**Formatting**:
- Strictly follow the JSON template below:
  {{
    "sensors": [
      {{
        "name": "Velodyne HDL-64E",
        "category": "Automotive LiDAR",
        "parts": ["Laser Module", "Spinning Mechanism"]
      }}
    ]
  }}

**Additional Instructions**:
- Only include a category if it contains sensors.
- Use consistent naming conventions for categories. Avoid redundancy or variations in names.
- If no sensors or parts are found in the text, return an empty JSON: {{}}.
- Do not include any additional explanations or information.

<text>
{text}
</text>


"""

In [58]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"
 
 
def call_model(
    prompt: str, response_format: ResponseFormat = ResponseFormat.TEXT
) -> str:
    response = ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        format="" if response_format == ResponseFormat.TEXT else "json",
    )
    return response["response"]

In [59]:
# Prepare the final prompt with the concatenated text

responses = []
CHUNK_SIZE = 1000
chunks = [retrieved_text[i:i + CHUNK_SIZE] for i in range(0, len(retrieved_text), CHUNK_SIZE)];
for i, chunk in enumerate(chunks):
        final_prompt = SUMMARIZE_PROMPT.format(text=chunk)
    # Send the final prompt to Ollama
        response = call_model(final_prompt)
        print(response)
        responses.append(response)



{
  "sensors": [
    {
      "name": "Livox Horizon",
      "category": "Automotive LiDAR",
      "parts": ["Laser Module", "Spinning Mechanism"]
    }
  ]
}
{
  "sensors": [
    {
      "name": "Velodyne HDL-64E",
      "category": "Automotive LiDAR",
      "parts": ["Laser Module", "Spinning Mechanism"]
    },
    {
      "name": "non-rotary LiDAR sensors (not specific brand)",
      "category": "Non-Rotary LiDAR",
      "parts": []
    }
  ]
}
{
  "sensors": [
    {
      "name": "Blickfeld Cube",
      "category": "Static Square Meter Reference Plane LiDAR",
      "parts": []
    },
    {
      "name": "Velarray",
      "category": "Automotive LiDAR",
      "parts": ["Laser Module", "Spinning Mechanism"]
    }
  ]
}
Here is the JSON output with the LiDAR sensors and their associated parts:

{
  "sensors": [
    {
      "name": "Velodyne Velarray",
      "category": "Automotive LiDAR",
      "parts": ["Laser Module", "Spinning Mechanism"]
    },
    {
      "name": "Robosense M1",
 

In [60]:
# Save responses after every chunk to ensure progress is retained
with open("responses.json", "w", encoding="utf-8") as f:
    json.dump(responses, f)

In [68]:
import json
import unicodedata
import re

# Function to clean and normalize strings
def clean_string(s):
    s = s.strip()  # Remove leading/trailing whitespace
    s = s.encode('utf-8').decode('unicode_escape')  # Decode Unicode escapes
    s = unicodedata.normalize('NFKC', s)  # Normalize special characters
    s = re.sub(r"[\s\_/]+", " ", s)  # Replace special separators with space
    return s.lower()

# Load responses and process them
with open("responses.json", "r", encoding="utf-8") as f:
    responses = json.load(f)

# Placeholder for final results and a set to track unique sensors
final_sensors = []
seen_sensors = set()

# Process responses into final_sensors
for response in responses:
    try:
        response_json = json.loads(response)

        if "sensors" in response_json and isinstance(response_json["sensors"], list):
            for sensor in response_json["sensors"]:
                sensor_name = clean_string(sensor.get("name", "Unnamed Sensor"))
                category_name = clean_string(sensor.get("category", "Uncategorized"))
                parts_list = sensor.get("parts", [])

                # Clean and sort parts to ensure consistent order
                cleaned_parts = sorted(clean_string(part) for part in parts_list)

                # Create a tuple to represent the sensor uniquely
                sensor_tuple = (sensor_name, category_name, tuple(cleaned_parts))

                # If the combination is not seen, add it to the list
                if sensor_tuple not in seen_sensors:
                    seen_sensors.add(sensor_tuple)
                    final_sensors.append({
                        "name": sensor_name,
                        "category": category_name,
                        "parts": cleaned_parts
                    })
    except json.JSONDecodeError as e:
        print(f"Error parsing response: {response}. Details: {e}")

# Save the cleaned and unique sensors to a new file
with open("final_sensors_with_parts5.json", "w", encoding="utf-8") as out_file:
    json.dump({"sensors": final_sensors}, out_file, ensure_ascii=False, indent=4)

print("Sensors processed, duplicates removed, and saved to final_sensors_with_parts5.json.")


Error parsing response: Here is the JSON output with the LiDAR sensors and their associated parts:

{
  "sensors": [
    {
      "name": "Velodyne Velarray",
      "category": "Automotive LiDAR",
      "parts": ["Laser Module", "Spinning Mechanism"]
    },
    {
      "name": "Robosense M1",
      "category": "Automotive LiDAR",
      "parts": []
    },
    {
      "name": "Livox Horizon",
      "category": "Automotive LiDAR",
      "parts": []
    }
  ]
}

Note that the category names are consistent and avoid redundancy. Only categories with sensors are included, and parts are only listed if they were mentioned in the text.. Details: Expecting value: line 1 column 1 (char 0)
Error parsing response: Since there are no specific LiDAR sensors or their associated parts mentioned in the text, I will return an empty JSON object.

```json
{
    "sensors": []
}
```

However, since you asked me to provide output based on your task, I can try to extract some information related to LiDAR sensors

In [70]:
from owlready2 import *

# Load the JSON file
with open("final_sensors_with_parts5.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Create a new ontology
onto = get_ontology("http://example.org/sensor_ontology.owl")

with onto:
    # Define basic classes and properties
    class Sensor(Thing):
        pass
    
    class Part(Thing):
        pass

    class has_part_directly(ObjectProperty):
        domain = [Sensor]
        range = [Part]
    # Define an annotation property for category
    class category(AnnotationProperty):
        pass

     # Process the JSON data
    part_classes = {}  # Cache for part classes to avoid duplicates

    for sensor_data in data["sensors"]:
        # Create or get the sensor class
        sensor_name = sensor_data["name"].replace(" ", "_")
        sensor_category = sensor_data["category"].replace(" ", "_")
        
        # Define the sensor class dynamically
        sensor_class = types.new_class(sensor_name, (Sensor,))
        
        # Assign category as an annotation
        sensor_class.category.append(sensor_category)

        # Add parts and relationships
        for part_name in sensor_data["parts"]:
            # Create or get the part class
            part_name_cleaned = part_name.replace(" ", "_")
            if part_name_cleaned not in part_classes:
                part_class = types.new_class(part_name_cleaned, (Part,))
                part_classes[part_name_cleaned] = part_class
            else:
                part_class = part_classes[part_name_cleaned]

            # Create the relationship
            sensor_class.is_a.append(has_part_directly.some(part_class))

# Save the ontology to a file
onto.save(file="sensor_ontology.owl", format="rdfxml")

print("Ontology created and saved as sensor_ontology.owl")

Ontology created and saved as sensor_ontology.owl


In [71]:

# Load the ontology
onto = get_ontology("sensor_ontology.owl").load()

# Print basic information to check loading
print(onto.base_iri)
print(list(onto.classes()))
print(list(onto.properties()))

http://example.org/sensor_ontology.owl#
[sensor_ontology.Sensor, sensor_ontology.Part, sensor_ontology.livox_horizon, sensor_ontology.laser_module, sensor_ontology.spinning_mechanism, sensor_ontology.velodyne_hdl-64e, sensor_ontology.non-rotary_lidar_sensors_(not_specific_brand), sensor_ontology.blickfeld_cube, sensor_ontology.velarray, sensor_ontology.innoviz_pro, sensor_ontology.velodyne_velarray, sensor_ontology.robosense_m1, sensor_ontology.blickfeld_cube_range, sensor_ontology.livox, sensor_ontology.scan_pattern_design]
[sensor_ontology.has_part_directly, sensor_ontology.category]
